In [ ]:
from tkinter import *
from tkinter import filedialog
import os
import tkinter as tk
from PIL import Image, ImageTk #python3-pil.imagetk, python-3-imaging-tk

def showimage():
    fln = filedialog.askopenfilename(initialdir=os.getcwd(), title="Select image file", filetype=(("JPG File", "*.jpg"), ("PNG file", "*.png"), ("All Files", "*.*")))
    img = Image.open(fln)
    img.thumbnail((350,350))
    img = ImageTk.PhotoImage(img)
    lbl.configure(image=img)
    lbl.image = img
    
    
root = Tk()

frm = Frame(root)
frm.pack(side=BOTTOM, padx=15, pady=15)

lbl = Label(root)
lbl.pack()

btn = Button(frm, text="Browse Images", command=showimage)
btn.pack(side=tk.LEFT)

btn2 = Button(frm,text="Exit", command=lambda: exit())
btn2.pack(side=tk.LEFT, padx=10)

root.title("Image Browser")
root.geometry("900x350")
root.mainloop()

